# Assembling the GSFLOW model

So far, two models, watershed model and groundwater model, are generated but are not connected. In this notebook, we will demonstrate how these two model can be connected to form the GSFLOW model. Note: To successfully run this exercise make sure to provide the correct paths for the PRMS folder and MODFLOW folder.

### Outline
* Assemble GSFLOW model from PRMS and MODFLOW
* Run the model and visualize the ouputs.


<img src = ".\figures\WorkFlow.png">


In [1]:
import os
from gsflow import GsflowModel
import flopy
import matplotlib.pyplot as plt
import pandas
import numpy as np
flopy.__version__

'3.2.10'

### (1) Load the PRMS model using pyGSFLOW
Notice that loaded control file has a "PRMS" mode.

In [2]:
control_file = r"./data/sagehen/prms/windows/sagehen.control"
gs = GsflowModel.load_from_file(control_file)

Control file is loaded
Working on loading PRMS model ...
Prms model loading ...
PRMS model loaded ...
Mode is set to PRMS only, loading PRMS model only


### (2) Load Modflow using pygsflow

pygsflow has a special module that loads Modflow models using FloPy, but corrects for loading and file writing issues specific to GSFLOW

In [3]:
from gsflow.modflow import Modflow

mf = Modflow.load(r"./data/sagehen/modflow/saghen.nam")

   loading iuzfbnd array...
   loading irunbnd array...
   loading vks array...
   loading eps array...
   loading thts array...
stress period 1:
   loading finf array...
stress period 2:


### (3) Add Modflow object to pyGSFLOW

In [4]:
print(mf.get_package_list())
gs.mf = mf

['DIS', 'BAS6', 'UPW', 'SFR', 'OC', 'UZF', 'NWT']


### (4) Change model mode and  synchronize PRMS and MODFLOW times

In [5]:
gs.control.set_values(name="model_mode", values=['GSFLOW'])
gs.control.set_values(name="start_time", values=[1982, 8, 1, 0, 0, 0])
gs.control.set_values(name='end_time', values=[1997, 3, 31, 0, 0, 0])
gs.control.set_values(name='modflow_time_zero', values=[1982, 8, 1, 0, 0, 0])
gs.control.set_values(name='print_debug', values = [1])

c:\users\jlarsen\desktop\usgs-pygsflow\trunk\gsflow\param_base.py:114: UserWarning: The record does not exist modflow_time_zero
  warnings.warn(err, UserWarning)


### (5) Generate GSFLOW input files in the desired workspace.  

In [6]:
gs.write_input(workspace=r"./data/temp", basename='saghen_new')

Writing the project files .....

changing model workspace...
   C:\Users\jlarsen\Desktop\usgs-pygsflow\trunk\examples\data\temp
Control file is written...
Parameters files are written...
Data file is written...
Modflow files are written...


In [7]:
gs.gsflow_exe = os.path.abspath(r"../bin/gsflow.exe")


### Running GSFLOW will take about 5 minutes....

In [8]:
gs.run_model()

pyGSFLOW is using the following executable to run the model: C:\Users\jlarsen\Desktop\usgs-pygsflow\trunk\bin\gsflow.exe

(base) C:\Users\jlarsen\Desktop\usgs-pygsflow\trunk\examples\data\temp>C:\Users\jlarsen\Desktop\usgs-pygsflow\trunk\bin\gsflow.exe C:\Users\jlarsen\Desktop\usgs-pygsflow\trunk\examples\data\temp\saghen_new_cont.control 


                          U.S. Geological Survey
        Coupled Groundwater and Surface-water FLOW model (GSFLOW)
                         Version 1.2.2 02/23/2018

    An integration of the Precipitation-Runoff Modeling System (PRMS)
    and the Modular Groundwater Model (MODFLOW-NWT and MODFLOW-2005)


                               MODFLOW-NWT 
  U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUNDWATER-FLOW MODEL
                         WITH NEWTON FORMULATION
                         VERSION 1.1.3, 8/01/2017 
              BASED ON MODFLOW-2005 VERSION 1.11.0 08/08/2013

 WARNING, map_results requested with nmapOutVars equal 0
 no map_res

(True, [])

# Results Visualization

* GSFLOW budget: Dailey flow rates from/to all the integrated hydrologic model compartments. 
* PRMS output: PRMS Water-Budget, Statistic Variables, and PRMS Animation Variables. 
* Modflow outputs (heads & budget)


### GSFLOW basin budget. 
GSFLOW saves the dialey flow rates (and volumes) for all the integrated hydrologic model compartments. The results are saved in a Comma-separated values (CSV) file, which can efficiently imported and visualized using Pandas.

In [9]:
# The filename can be found in the control object under the name "csv_output_file"
csv_file = gs.control.get_values("csv_output_file")[0]

# Import results into Pandas
basin_bd = pandas.read_csv(csv_file)

# Show the first 10 lines of the budget in the notebook
basin_bd.head(10)

FileNotFoundError: File b'saghen_new_csv_output.csv' does not exist

In [ ]:
# To demonstrate plotting a budget component, let us plot the daily recharge values
basin_bd.plot(x='Date', y = 'RechargeUnsat2Sat_Q')

Importing the budget into Pandas allows us to access all of the powerful built in operations that come with Pandas

For example, let's plot the annual precipitation

In [ ]:
basin_bd['Date'] = pandas.to_datetime(basin_bd['Date'])
Budget_by_year = basin_bd.groupby(basin_bd.Date.dt.year).sum()


In [ ]:
Budget_by_year['StreamOut_Q'].plot() ; plt.title("Annual Stream Outflow")

In [ ]:
Budget_by_month = basin_bd.groupby(basin_bd.Date.dt.month).mean()
Budget_by_month['StreamOut_Q'].plot(); plt.title("Monthly Stream Outflow")

### PRMS Statistic Variables
This following method can be used to import statVar file into Pandas.

In [ ]:
# Load the results
gs.prms.get_StatVar()

In [ ]:
# To plot all results, we can use the following method...
gs.prms.stat.plot()

In [ ]:
## Importing the animation is not supported yet. In the future, a method will be added to generate vedio from animation file.

# MODFLOW Results

In [ ]:
mf = gs.mf
mf

### Hydraulic Head Maps

In [ ]:
# List Modflow output files
mf.output_fnames

In [ ]:
# Obtain heads output file name
head_file = os.path.join(mf.model_ws, mf.output_fnames[2])
head_file

cbc = os.path.join(mf.model_ws, mf.output_fnames[0])

In [ ]:
# load the head into the hds object
hds = flopy.utils.HeadFile(head_file)
cbc = flopy.utils.CellBudgetFile(cbc)

In [ ]:
# Since the head is saved at different time points, we first need to fid these times
print(cbc.textlist)
fff = cbc.get_data(text="FLOW FRONT FACE", full3D=True)[-1]
frf = cbc.get_data(text="FLOW RIGHT FACE", full3D=True)[-1]
flf = cbc.get_data(text="FLOW LOWER FACE", full3D=True)[-1]
# hds.get_kstpkper()

In [ ]:
# To get the data at any time point, 
head = hds.get_data(kstpkper= (729, 1))
head[head<0] = np.nan

In [ ]:
# Now we can plot the head map
plt.figure(figsize=(17,6));
plt.subplot(1,2,1)
plt.contourf(head[0,:,:],50, cmap = 'jet');plt.gca().invert_yaxis(); plt.colorbar(); plt.title("Head for Layer 1")
plt.axis('equal')
plt.subplot(1,2,2)
plt.contourf(head[1,:,:],50, cmap = 'jet');plt.gca().invert_yaxis(); plt.colorbar(); plt.title("Head for Layer 2")
plt.axis('equal')

## Head Cross-section

In [ ]:
head = hds.get_data()
levels = np.arange(1830, 2505, 10)
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(2, 1, 1)
ax.set_title('Head Cross-section')
modelxsect = flopy.plot.ModelCrossSection(model=mf, line={'Column': 50})
ct = modelxsect.contour_array(head, masked_values=[999.], head=head, levels=levels)
patches = modelxsect.plot_ibound(head=head)
wt = modelxsect.plot_surface(head, masked_values=[999.], color='blue', lw=1)
linecollection = modelxsect.plot_grid()
disc = modelxsect.plot_discharge(frf, fff, flf)

### Heads Timeseries
Sometimes we are interested more in a head change with time at a certain point...

In [ ]:
plt.figure(figsize=(10,7)); 

# The point should be specified as  (layer, row, column)
point = (0,50,50)

# Extract time series at this point
h_ts = hds.get_ts(idx=point)
plt.plot(h_ts[2:,0], h_ts[2:,1], 'b', label = "Layer 1")


# let us do the same thing for the second layer and compare the results ...
point = (1,50,50)
h_ts = hds.get_ts(idx=point)
plt.plot(h_ts[2:,0], h_ts[2:,1], 'r', label = "Layer 2")
plt.xlabel("Time (Days)")
plt.ylabel("Heads (m)")
plt.legend()

### Modflow Budget 

In [ ]:
# Obtain heads output file name
cbc_file = os.path.join(mf.model_ws, mf.output_fnames[0])
cbc_file

In [ ]:
# Load Budget
cbc = flopy.utils.CellBudgetFile(cbc_file)

In [ ]:
# list the time points where budget result is available
cbc.get_kstpkper()

In [ ]:
# list budget items that exist in the budget file
cbc.textlist

In [ ]:
## Plot 
plt.figure(figsize=(15,10)); 
frf = cbc.get_data(full3D=True, kstpkper= (5355, 1),text='FLOW RIGHT FACE ')
plt.subplot(2,2,1)
plt.contourf(frf[0][0,:,:]);plt.gca().invert_yaxis(); plt.colorbar()
plt.title('FLOW RIGHT FACE ')

fff = cbc.get_data(full3D=True, kstpkper= (5355, 1),text= 'FLOW FRONT FACE ')
plt.subplot(2,2,2)
plt.contourf(fff[0][0,:,:]);plt.gca().invert_yaxis(); plt.colorbar()
plt.title('FLOW FRONT FACE ')


flf = cbc.get_data(full3D=True, kstpkper= (5355, 1),text= 'FLOW LOWER FACE ')
plt.subplot(2,2,3)
plt.contourf(flf[0][0,:,:]);plt.gca().invert_yaxis(); plt.colorbar()
plt.title('FLOW LOWER FACE ')


xx = cbc.get_data(full3D=True, kstpkper= (5355, 1),text=   '  STREAM LEAKAGE' )
plt.subplot(2,2,4)
plt.contourf(xx[0][0,:,:]);plt.gca().invert_yaxis(); plt.colorbar()
plt.title('  STREAM LEAKAGE')
plt.show()

mm = flopy.plot.ModelMap(model=mf, layer=0)
mm.plot_discharge(frf[0], flf[0], istep=1, jstep=1)

### Reading information from the list file

In [ ]:
list_file = mf.name + ".list"
list_file

In [ ]:
mf_list = flopy.utils.MfListBudget(list_file)
incrementaldf, cumulativedf = mf_list.get_dataframes()

In [ ]:
incrementaldf

In [ ]:
incrementaldf['STORAGE_IN'].plot(label = 'Storage_in')
incrementaldf['STORAGE_OUT'].plot(label = 'Storage_out')
plt.legend()

In [ ]:
incrementaldf['IN-OUT'].plot()

In [ ]:
!jupyter nbconvert --to script Assemble_GSFLOW_model.ipynb